<a href="https://colab.research.google.com/github/rahilv99/Microbial_Optimization_Algorithim/blob/main/You_Lab_Growth_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TO DO LIST
- ensure the model is designed to display best input factors
  - displaying known input factors right now, should display predicted factors
- change optimization model to better model the context of the problem
  - loss fcn is MSE right now
  - implement multiple models, test and use best one
- find what training factors to evaluate on (not MSE)


Import libraries

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

Process actual data from csv

In [ ]:
# Load dataset
df_growth = pd.read_csv('')

# print(df_bank.info())
print('Shape of dataframe:', df_growth.shape)
df_growth.head()

Read growth data distrubution
- May need to normalize growth factors to be 0 to 1 from min to max

In [ ]:
description = df_growth['numbers'].describe()

print(f"Range of 'numbers' column: {description['min']} - {description['max']}")
print(f"Standard deviation: {description['std']}")
print(f"Mean: {description['mean']}")

Split into factor and target data

In [ ]:
input_data = df_growth.drop('growth factor', axis=1)
target_data = df_growth['growth factor']

Create dummy data for demo

In [ ]:
np.random.seed(42)
num_samples = 1000
input_data = np.random.rand(num_samples, 5)  # 5 input factors
target_data = np.random.rand(num_samples, 1)  # Growth factor (target variable)

Split into train and test data

In [ ]:
# Split data into training and validation sets using scikit-learn
input_train, input_val, target_train, target_val = train_test_split(
    input_data, target_data, test_size=0.2, random_state=42
)

# Standardize input data using scikit-learn
scaler = StandardScaler()
input_train_scaled = scaler.fit_transform(input_train)
input_val_scaled = scaler.transform(input_val)

Create Optimization Model
- Can change # of nodes (increase for high bias, decrease for high variance)
- lambda in L2 (increase for high variance, decrease for high bias)
- Can add more layers (increase for high bias, decrease for high variance)
- May need to import Adam as tf.keras.optimizers.Adam(learning_rate=1e^-3)

In [ ]:
# Define the optimization model using TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(5,), name='input_layer'),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='hidden_layer_1'),
    tf.keras.layers.Dense(1, activation='linear', kernel_regularizer=tf.keras.regularizers.l2(0.01), name='output_layer')
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

Train Model

In [ ]:
# Train the model using scikit-learn compatible fit method
history = model.fit(input_train_scaled, target_train, epochs=50, batch_size=32,
                    validation_data=(input_val_scaled, target_val))

Evaluate Model

In [ ]:
# Evaluate the model using test data
evaluation_result = model.evaluate(input_val_scaled, target_val)
print(f"Evaluation Result: {evaluation_result}")

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Cross Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Use Model
- Bad runtime complexity, can edit later

In [ ]:
# Find the input factors that result in the highest growth factor
best_input_factors = None
best_growth_factor = -1

for i in range(len(input_val_scaled)):
    current_input_factors = input_val_scaled[i:i+1, :]
    current_growth_factor = model.predict(current_input_factors)[0][0]

    if current_growth_factor > best_growth_factor:
        best_growth_factor = current_growth_factor
        best_input_factors = current_input_factors

# Inverse transform to get the original scale of input factors
best_input_factors_original_scale = scaler.inverse_transform(best_input_factors)

print("Best Input Factors (Original Scale):", best_input_factors_original_scale.flatten())
print("Corresponding Growth Factor:", best_growth_factor)